In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectionSlider

from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

#from stumpy import stump, mstump

import lightgbm as lgbm
from lightgbm import LGBMClassifier

import optuna

from preprocessing import create_tensor, agg_over_months, min_max_scaling, standard_scaling
from model_lgbm import train_lgbm, train_best_lgbm, lgbm_plot_evals, lgbm_f1, double_soft_f1_loss
from training import cv_split, study_summary
from utils import save_pred


plt.rcParams["figure.figsize"] = (12, 6)
#np.random.seed(0)

# Setup

In [2]:
train_df = pd.read_csv("./data/train.csv", index_col=0).drop_duplicates()
test_df = pd.read_csv("./data/test_nolabels.csv", index_col=0)

class0_idx = train_df.loc[train_df.LABELS == 0].index
class1_idx = train_df.loc[train_df.LABELS == 1].index
y1_weight = train_df.LABELS.sum() / train_df.shape[0]
# y1_weight = 0.6757894386583703

In [3]:
# Constants to select pandas columns efficiently
# all months abbreviations: jan, feb, mar, etc.
MONTHS = [m.lower() for m in pd.date_range(0, freq="M", periods=12).strftime("%b").to_list()]

# return all columns based on MONTH: train_df[COL_BY_MONTH["jan"]]
COL_BY_MONTH = {}
for month in MONTHS:
    COL_BY_MONTH[month] = [col for col in train_df.columns if month in col]

# all features
FEATURES = [col for col in train_df.columns if col != "LABELS"]
# all features with the month stripped: S2_B2_
COL_BASE = list({col[:-3] for col in FEATURES})
# return all columns based on FEATURE (e.g., across months): train_df[COL_BY_FEATURE["S2_B2_"]]
# alternatively, you can select the same df using: train_df[COL_BY_FEATURE[FEATURES[0]]]
COL_BY_FEATURE = {}
for feature in COL_BASE:
    COL_BY_FEATURE[feature] = [col for col in train_df.columns if feature in col]

# Preprocessing

In [4]:
agg_train = agg_over_months(train_df, freq=6)
agg_test = agg_over_months(test_df, freq=6)
scaled_train, scaled_test = standard_scaling(train_df, test_df)

In [5]:
pca = PCA(n_components = 20)
train_x = scaled_train[[c for c in train_df.columns if c != "LABELS"]]
projected_train = pd.DataFrame(pca.fit_transform(train_x), columns=[f"comp{i}" for i in range(20)])
projected_train[~np.isfinite(projected_train)] = 0
projected_train["LABELS"] = train_df.reset_index().LABELS

In [8]:
projected_test = pd.DataFrame(pca.transform(test_df), columns=[f"comp{i}" for i in range(20)])
projected_test[~np.isfinite(projected_test)] = 0

## optuna optimization

In [9]:
def objective(trial, train_df):
    # hyperparameter grid for Optuna to explore
    hyperparams = {
        "class_weight": "balanced", 
        "device_type": trial.suggest_categorical("device_type", ["gpu"]),
        # structure
        "max_depth": trial.suggest_int("max_depth", 3, 12, step=1),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=50),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        # accuracy
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_categorical("n_estimators", [500]),
        # overfitting
        #"max_bin": trial.suggest_int("max_bin", 200, 300),
        #"lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        #"lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        #"feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95, step=0.1),
    }
    
    # split features and labels
    x_df = train_df.loc[:, ~train_df.columns.isin(["LABELS"])]
    y_df = train_df.loc[:, "LABELS"]
    # create cross-validation folds object
    kfold_cv = StratifiedKFold(n_splits=5, shuffle=True)
    
    # hold the best score of each fold
    cv_scores = []
    for idx, (train_idx, test_idx) in enumerate(kfold_cv.split(x_df, y_df)):
        # train-validation split for each fold
        x_train, x_test = x_df.iloc[train_idx], x_df.iloc[test_idx]
        y_train, y_test = y_df.iloc[train_idx], y_df.iloc[test_idx]
        
        # train classifier with optuna hyperparameters
        clf = LGBMClassifier(objective="binary", **hyperparams)
        clf.fit(
            x_train,
            y_train,
            eval_set=[(x_test, y_test)],
            eval_metric=["logloss", "double_soft_f1_loss"],
        )
        
        # store the best score to the CV score
        best_score = clf.best_score_["valid_0"]["binary_logloss"]#["double_soft_f1_loss"]
        cv_scores.append(best_score)
    
    # return the average best score across CV folds
    return np.mean(cv_scores)

In [10]:
# create optuna study
study = optuna.create_study(direction="minimize", study_name="lgbm_split")
optimize = lambda trial: objective(trial, projected_train)

[I 2021-12-12 18:07:22,792] A new study created in memory with name: lgbm_split


In [11]:
# launch optuna study
study.optimize(optimize, n_trials=20)

C:\Users\TJ\miniconda3\envs\kaggle2\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [20, 3000] and step=50, but the range is not divisible by `step`. It will be replaced by [20, 2970].
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=8200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8200
[LightGBM] [Warning] min_gain_to_split is set=13.9303815139525, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=13.9303815139525
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] lambda_l2 is set=75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=75
[1]	valid_0's binary_logloss: 0.67392
[2]	valid_0's binary_logloss: 0.659291
[3]	valid_0's binary_logloss: 0.646892
[4]	valid_0's binary_logloss: 0.638976
[5]	valid_0's binary_logloss: 0.629919
[6]	valid_0's binary_logloss: 0.623536
[7]	valid_0's binary_logloss: 0.61726
[8]	valid_0's binary_logloss: 0.61138
[9]	valid_0's binary_logloss: 0.607726
[10]	valid_0's binary_logloss: 0.604268
[11]	valid_0's binary_logloss: 0.601178
[12]	valid_0's binary_logloss: 0.597886
[13]	valid_0's binary_logloss: 0.595314
[14]	valid_0's 

[I 2021-12-12 18:07:26,984] Trial 0 finished with value: 0.5544291907605169 and parameters: {'device_type': 'gpu', 'max_depth': 7, 'num_leaves': 670, 'min_data_in_leaf': 8200, 'learning_rate': 0.14272374805238366, 'n_estimators': 500, 'lambda_l1': 5, 'lambda_l2': 75, 'min_gain_to_split': 13.9303815139525}. Best is trial 0 with value: 0.5544291907605169.


[279]	valid_0's binary_logloss: 0.55095
[280]	valid_0's binary_logloss: 0.55095
[281]	valid_0's binary_logloss: 0.55095
[282]	valid_0's binary_logloss: 0.55095
[283]	valid_0's binary_logloss: 0.55095
[284]	valid_0's binary_logloss: 0.55095
[285]	valid_0's binary_logloss: 0.55095
[286]	valid_0's binary_logloss: 0.55095
[287]	valid_0's binary_logloss: 0.55095
[288]	valid_0's binary_logloss: 0.55095
[289]	valid_0's binary_logloss: 0.55095
[290]	valid_0's binary_logloss: 0.55095
[291]	valid_0's binary_logloss: 0.55095
[292]	valid_0's binary_logloss: 0.55095
[293]	valid_0's binary_logloss: 0.55095
[294]	valid_0's binary_logloss: 0.55095
[295]	valid_0's binary_logloss: 0.55095
[296]	valid_0's binary_logloss: 0.55095
[297]	valid_0's binary_logloss: 0.55095
[298]	valid_0's binary_logloss: 0.55095
[299]	valid_0's binary_logloss: 0.55095
[300]	valid_0's binary_logloss: 0.55095
[301]	valid_0's binary_logloss: 0.55095
[302]	valid_0's binary_logloss: 0.55095
[303]	valid_0's binary_logloss: 0.55095


C:\Users\TJ\miniconda3\envs\kaggle2\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [20, 3000] and step=50, but the range is not divisible by `step`. It will be replaced by [20, 2970].
  warnings.warn(


...

[218]	valid_0's binary_logloss: 0.516274
[219]	valid_0's binary_logloss: 0.516274
[220]	valid_0's binary_logloss: 0.516274
[221]	valid_0's binary_logloss: 0.516274
[222]	valid_0's binary_logloss: 0.516274
[223]	valid_0's binary_logloss: 0.516274
[224]	valid_0's binary_logloss: 0.516274
[225]	valid_0's binary_logloss: 0.516274
[226]	valid_0's binary_logloss: 0.516274
[227]	valid_0's binary_logloss: 0.516274
[228]	valid_0's binary_logloss: 0.516274
[229]	valid_0's binary_logloss: 0.516274
[230]	valid_0's binary_logloss: 0.516274
[231]	valid_0's binary_logloss: 0.516274
[232]	valid_0's binary_logloss: 0.516274
[233]	valid_0's binary_logloss: 0.516274
[234]	valid_0's binary_logloss: 0.516274
[235]	valid_0's binary_logloss: 0.516274
[236]	valid_0's binary_logloss: 0.516274
[237]	valid_0's binary_logloss: 0.516274
[238]	valid_0's binary_logloss: 0.516274
[239]	valid_0's binary_logloss: 0.516274
[240]	valid_0's binary_logloss: 0.516274
[241]	valid_0's binary_logloss: 0.516274
[242]	valid_0's 

C:\Users\TJ\miniconda3\envs\kaggle2\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [20, 3000] and step=50, but the range is not divisible by `step`. It will be replaced by [20, 2970].
  warnings.warn(


[1]	valid_0's binary_logloss: 0.690985
[2]	valid_0's binary_logloss: 0.688872
[3]	valid_0's binary_logloss: 0.686808
[4]	valid_0's binary_logloss: 0.684786
[5]	valid_0's binary_logloss: 0.682814
[6]	valid_0's binary_logloss: 0.680878
[7]	valid_0's binary_logloss: 0.678984
[8]	valid_0's binary_logloss: 0.67714
[9]	valid_0's binary_logloss: 0.675349
[10]	valid_0's binary_logloss: 0.673572
[11]	valid_0's binary_logloss: 0.671845
[12]	valid_0's binary_logloss: 0.670144
[13]	valid_0's binary_logloss: 0.668502
[14]	valid_0's binary_logloss: 0.666871
[15]	valid_0's binary_logloss: 0.665275
[16]	valid_0's binary_logloss: 0.663714
[17]	valid_0's binary_logloss: 0.662182
[18]	valid_0's binary_logloss: 0.660704
[19]	valid_0's binary_logloss: 0.659238
[20]	valid_0's binary_logloss: 0.6578
[21]	valid_0's binary_logloss: 0.656392
[22]	valid_0's binary_logloss: 0.655034
[23]	valid_0's binary_logloss: 0.653689
[24]	valid_0's binary_logloss: 0.652395
[25]	valid_0's binary_logloss: 0.651128
[26]	valid_0

[I 2021-12-12 18:08:15,887] Trial 19 finished with value: 0.5328984738477942 and parameters: {'device_type': 'gpu', 'max_depth': 9, 'num_leaves': 2720, 'min_data_in_leaf': 3600, 'learning_rate': 0.011612212124944205, 'n_estimators': 500, 'lambda_l1': 25, 'lambda_l2': 40, 'min_gain_to_split': 2.9894426838370376}. Best is trial 13 with value: 0.4916794108004551.


[462]	valid_0's binary_logloss: 0.529424
[463]	valid_0's binary_logloss: 0.529357
[464]	valid_0's binary_logloss: 0.529291
[465]	valid_0's binary_logloss: 0.529203
[466]	valid_0's binary_logloss: 0.529149
[467]	valid_0's binary_logloss: 0.529085
[468]	valid_0's binary_logloss: 0.528989
[469]	valid_0's binary_logloss: 0.528951
[470]	valid_0's binary_logloss: 0.528904
[471]	valid_0's binary_logloss: 0.52885
[472]	valid_0's binary_logloss: 0.528809
[473]	valid_0's binary_logloss: 0.528739
[474]	valid_0's binary_logloss: 0.528647
[475]	valid_0's binary_logloss: 0.528561
[476]	valid_0's binary_logloss: 0.528482
[477]	valid_0's binary_logloss: 0.528438
[478]	valid_0's binary_logloss: 0.528377
[479]	valid_0's binary_logloss: 0.528299
[480]	valid_0's binary_logloss: 0.528215
[481]	valid_0's binary_logloss: 0.528158
[482]	valid_0's binary_logloss: 0.528098
[483]	valid_0's binary_logloss: 0.528063
[484]	valid_0's binary_logloss: 0.527989
[485]	valid_0's binary_logloss: 0.527921
[486]	valid_0's b

In [12]:
study_summary(study)

Study:  lgbm_split

Number of finished trials: 20
Best trial:
  Validation score: 0.4916794108004551
  Params: 
    device_type: gpu
    max_depth: 12
    num_leaves: 2520
    min_data_in_leaf: 300
    learning_rate: 0.2011884316999342
    n_estimators: 500
    lambda_l1: 30
    lambda_l2: 35
    min_gain_to_split: 2.7591039026962063


In [13]:
# pass the optuna best parameters as hyperparameters
best_clf = train_best_lgbm(projected_train, study.best_params)

In [14]:
best_clf.__dict__

{'boosting_type': 'gbdt',
 'objective': 'binary',
 'num_leaves': 2520,
 'max_depth': 12,
 'learning_rate': 0.2011884316999342,
 'n_estimators': 500,
 'subsample_for_bin': 200000,
 'min_split_gain': 0.0,
 'min_child_weight': 0.001,
 'min_child_samples': 20,
 'subsample': 1.0,
 'subsample_freq': 0,
 'colsample_bytree': 1.0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'random_state': None,
 'n_jobs': -1,
 'silent': 'warn',
 'importance_type': 'split',
 '_Booster': <lightgbm.basic.Booster at 0x1e36012d520>,
 '_evals_result': None,
 '_best_score': defaultdict(collections.OrderedDict, {}),
 '_best_iteration': None,
 '_other_params': {'device_type': 'gpu',
  'min_data_in_leaf': 300,
  'lambda_l1': 30,
  'lambda_l2': 35,
  'min_gain_to_split': 2.7591039026962063},
 '_objective': 'binary',
 'class_weight': None,
 '_class_weight': None,
 '_class_map': {0.0: 0, 1.0: 1},
 '_n_features': 20,
 '_n_features_in': 20,
 '_classes': array([0., 1.]),
 '_n_classes': 2,
 'device_type': 'gpu',
 'min_data_in_leaf

In [15]:
def standard_scaling(train_df, test_df):
    standard_scaler = StandardScaler()
    feature_cols = [col for col in train_df.columns if col != "LABELS"]
    
    train_df[feature_cols] = standard_scaler.fit_transform(train_df[feature_cols])
    test_df[feature_cols] = standard_scaler.transform(test_df[feature_cols])

    return train_df, test_df

In [16]:
agg_test = agg_over_months(test_df, freq=3)
scaled_train, scaled_test = standard_scaling(agg_train, agg_test)

KeyError: "None of [Index(['NDVI_06_mean', 'NDVI_612_mean', 'S2_B4_06_mean', 'S2_B4_612_mean',\n       'S2_B2_06_mean', 'S2_B2_612_mean', 'S2_B12_06_mean', 'S2_B12_612_mean',\n       'ERA5_total_precipitation_06_mean', 'ERA5_total_precipitation_612_mean',\n       'topo_elevation_06_mean', 'topo_elevation_612_mean', 'S2_B11_06_mean',\n       'S2_B11_612_mean', 'S2_B9_06_mean', 'S2_B9_612_mean', 'S1_VV_06_mean',\n       'S1_VV_612_mean', 'S1_VH_06_mean', 'S1_VH_612_mean', 'S2_B6_06_mean',\n       'S2_B6_612_mean', 'S2_B7_06_mean', 'S2_B7_612_mean',\n       'ERA5_temperature_2m_06_mean', 'ERA5_temperature_2m_612_mean',\n       'topo_slope_06_mean', 'topo_slope_612_mean', 'S2_B5_06_mean',\n       'S2_B5_612_mean', 'S2_B8A_06_mean', 'S2_B8A_612_mean', 'S2_B3_06_mean',\n       'S2_B3_612_mean', 'S2_B8_06_mean', 'S2_B8_612_mean'],\n      dtype='object')] are in the [columns]"

In [69]:
preds = best_clf.predict(agg_test)

# Save prediction

In [71]:
save_pred(projected_test, preds, "7")